In [3]:
import networkx as nx
import numpy as np
from torch_geometric.utils import to_networkx

from motclass import MotDataset, build_graph
from utilities import get_best_device

device = get_best_device()

# mot20_path = "/media/dmmp/vid+backup/Data/MOT20/images"
mot20_path = "data/MOT20"


[INFO] Using MPS.


In [4]:
def draw_graph(graph, track):
	G = to_networkx(graph, to_undirected=True)

	n_sum = [0] + list(np.cumsum(track.n_nodes))

	for i, n in enumerate(track.n_nodes):
		for j in range(n):
			G.nodes[j + n_sum[i]]['level'] = i

	pos = nx.multipartite_layout(G, subset_key="level")

	nx.draw(G, pos=pos, with_labels=True)


In [33]:
mot20 = MotDataset(mot20_path, 'train', subtrack_len=15, device=device)

track = mot20[1]

graph = build_graph(**track.get_data(), device=device)

# draw_graph(graph, track)


[INFO] Subtrack #1 (track #0 (frames 30 - 45)
[INFO] Track has 15 frames


Reading frame data/MOT20/train/MOT20-03/img1/000045.jpg: 100%|██████████| 15/15 [00:00<00:00, 29.53it/s]
Linking nodes: 100%|██████████| 15/15 [00:00<00:00, 43.32it/s]


[INFO] 15 frames
[INFO] 888 total nodes
[INFO] 735886 total edges


---

In [59]:
mots = ["MOT15", "MOT17", "MOT20"]

for mot in mots:

	sizes = list()

	mot20 = MotDataset(f'data/{mot}', 'train', linkage_window="ADJACENT", device=device)

	for track in mot20:
		sizes.append(track._get_avg_size())

	# get avg for each element in the tuple
	avg_sizes = list(map(np.mean, zip(*sizes)))
	avg_sizes = [round(x, 2) for x in avg_sizes]

	print(mot, avg_sizes)

# ~avg across all MOTs: 70x170


MOT15 [59.55, 136.0]
MOT17 [83.52, 234.14]
MOT20 [67.75, 154.0]
